In [1]:
import subprocess
import logging
import os
import numpy as np
import random
import shutil
import platform
import sys
from typing import Union, List, Optional

In [104]:
# Type hinting for custom/user-defined classes:
# https://stackoverflow.com/questions/44664040/type-hints-with-user-defined-classes
# https://stackoverflow.com/questions/33387042/type-hinting-argument-of-type-class
# https://docs.python.org/3/library/typing.html
# https://python-type-checking.readthedocs.io/en/latest/types.html
# https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html

In [114]:
class DependencyError(Exception):
    pass

In [8]:
class Command(object):
    '''
    Creates a command and an empty command list for UNIX command line programs/applications. Primary use and
    use-cases are intended for the subprocess module and its associated classes (i.e. Popen/call/run).
    
    Attributes (class and instance attributes):
        command (instance): Command to be performed on the command line.
        cmd_list (instance): Mutable list that can be appended to.
    
    Modules/Packages required:
        - os
        - logging
        - subprocess
    '''

    def __init__(self,
                 command: str) -> list:
        '''
        Init doc-string for Command class. Initializes a command to be used on UNIX command line.
        The input argument is a command (string), and a mutable list is returned (, that can later
        be appended to).
        
        Usage:
            echo = Command("echo")
            echo.cmd_list.append("Hi!")
            echo.cmd_list.append("I have arrived!")
        
        Arguments:
            command (string): Command to be used. Note: command used must be in system path
        Returns:
            cmd_list (list): Mutable list that can be appended to.
        '''
        self.command = command
        self.cmd_list = [f"{self.command}"]
        
    def check_dependency(self,
                 err_msg: Optional[str] = None):
        '''doc-string'''
        if not shutil.which(self.command):
            if err_msg:
                print(f"\n \t {err_msg} \n")
            else:
                print(f"\n \t The required dependency {self.command} is not installed or in the system path. \n")
            raise DependencyError("Command executable not found in system path.")
        
    def log(self,
            log_file: str = "log_file.log",
            log_cmd: str = ""):
        '''
        Log function for logging commands and messages to some log file.
        
        Usage:
            # Initialize the `log` function command
            log_msg = Command("log")
            
            # Specify output file and message
            log_msg.log("sub.log","test message 1")
            
            # Record message, however - no need to re-initialize `log` funcion command or log output file
            log_msg.log("test message 2")
        
        NOTE: The input `log_file` only needs to be specified once. Once specified,
            this log is written to each time this or the `run` function is invoked.
        
        Arguments:
            log_file(file): Log file to be written to. 
            log_cmd(str): Message to be written to log file
        '''
        
        # Set-up logging to file
        logging.basicConfig(level=logging.INFO,
                            format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                            datefmt='%d-%m-%y %H:%M:%S',
                            filename=log_file,
                            filemode='a')
        
        # Define a Handler which writes INFO messages or higher to the sys.stderr
        console = logging.StreamHandler()
        console.setLevel(logging.INFO)
        
        # Add the handler to the root logger
        logging.getLogger().addHandler(console)
        
        # Define logging
        logger = logging.getLogger(__name__)
        
        # Log command/message
        logger.info(f"{log_cmd}")
        
    def run(self,
            log_file: str = "",
            debug: bool = False,
            dryrun: bool = False,
            env: dict = {},
            stdout: str = "",
            shell: bool = False
           ) -> List[Union[int,str]]:
        '''
        Uses python's built-in subprocess class to execute (run) a command from an input command list.
        The standard output and error can optionally be written to file.
        
        Usage:
            echo.run() # This will return tuple (returncode,log,None,None), but will echo "Hi!" to screen.
            
        NOTE: 
            - The contents of the 'stdout' output file will be empty if 'shell' is set to True.
            - Once the log file name 'log_file' has been set, that value is stored and cannot be changed.
                - This log file will continue to be appended to for each invocation of this class.
        Arguments:
            log_file(file): Output log file name.
            debug(bool): Sets logging function verbosity to DEBUG level
            dryrun(bool): Dry run -- does not run task. Command is recorded to log file.
            env(dict): Dictionary of environment variables to add to subshell.
            stdout(file): Output file to write standard output to.
            shell(bool): Use shell to execute command.
        Returns:
            p.returncode(int): Return code for command execution should the 'log_file' option be used.
            log_file(file): Output log file with appended information should the 'log_file' option be used.
            stdout(file): Standard output writtent to file should the 'stdout' option be used.
            stderr(file): Standard error writtent to file should the 'stdout' option be used.
        '''
        
        # Define logging
        logger = logging.getLogger(__name__)
        cmd = ' '.join(self.cmd_list) # Join list for logging purposes
        
        if debug:
            logger.debug(f"Running: {cmd}")
        else:
            logger.info(f"Running: {cmd}")
        
        if dryrun:
            logger.info("Performing command as dryrun")
            return 0
        
        # Define environment variables
        merged_env = os.environ
        if env:
            merged_env.update(env)
        
        # Execute/run command
        p = subprocess.Popen(self.cmd_list,shell=shell,env=merged_env,
                        stdout=subprocess.PIPE,stderr=subprocess.PIPE)

        # Write log files
        out,err = p.communicate()
        out = out.decode('utf-8')
        err = err.decode('utf-8')

        # Write std output/error files
        if stdout:
            stderr = os.path.splitext(stdout)[0] + ".err"
            with open(stdout,"a") as f_out:
                with open(stderr,"a") as f_err:
                    f_out.write(out)
                    f_err.write(err)
                    f_out.close(); f_err.close()
        else:
            stdout = None
            stderr = None

        if p.returncode:
            logger.error(f"command: {cmd} \n Failed with returncode {p.returncode}")

        if len(out) > 0:
            if debug:
                logger.debug(out)
            else:
                logger.info(out)

        if len(err) > 0:
            if debug:
                logger.info(err)
            else:
                logger.warning(err)
        return p.returncode,log_file,stdout,stderr

In [2]:
class File(object):
    '''
    class description
    '''
    
    file = ""
    
    def __init__(self,
                 file: str) -> File:
        '''Init doc-string'''
        self.file = file
        
    def touch(self) -> File:
        '''doc-string'''
        with open(self.file,'w') as tmp_file:
            pass
        return self.file
    
    def abs_path(self) -> str:
        '''doc-string'''
        if os.path.exists(self.file):
            return os.path.abspath(self.file)
        else:
            self.touch()
            file_path = os.path.abspath(self.file)
            os.remove(self.file)
            return file_path
    
    def rm_ext(self,
              ext: str = "") -> str:
        '''doc-string'''
        self.ext = ext
        if self.ext is None:
            return self.file[:-(4)]
        else:
            ext_num = len(self.ext)
            return self.file[:-(ext_num)]
        
    def write_txt(self,
                 txt: str = "") -> File:
        '''doc-string'''
        with open(self.file,"a") as tmp_file:
            tmp_file.write(txt)
            tmp_file.close()
        return self.file

In [18]:
class TmpDir(object):
    '''
    class description
    '''
    
    # Set parent tmp directory, as tmp_dir is overwritten
    tmp_dir = ""
    parent_tmp_dir = ""
    
    def __init__(self,
                tmp_dir: str,
                use_cwd: bool = False):
        '''init doc-string'''
        _n = 10000 # maximum N for random number generator
        tmp_dir = os.path.join(tmp_dir,'tmp_dir_' + 
                               str(random.randint(0,_n)))
        self.tmp_dir = tmp_dir
        self.parent_tmp_dir = os.path.dirname(self.tmp_dir)
        if use_cwd:
            _cwd = os.getcwd()
            self.tmp_dir = os.path.join(_cwd,self.tmp_dir)
            self.parent_tmp_dir = os.path.dirname(self.tmp_dir)
        
    def mk_tmp_dir(self):
        '''doc-string'''
        if not os.path.exists(self.tmp_dir):
            return os.makedirs(self.tmp_dir)
        else:
            print("Temporary directory already exists")
        
    def rm_tmp_dir(self,
                  rm_parent: bool = False):
        '''doc-string'''
        if rm_parent and os.path.exists(self.parent_tmp_dir):
            return shutil.rmtree(self.parent_tmp_dir)
        elif os.path.exists(self.tmp_dir):
            return shutil.rmtree(self.tmp_dir)
        else:
            print("Temporary directory does not exist")
    
    class TmpFile(File):
        '''
        class description - make this a child class of TmpDir
        '''
        
        tmp_file = ""

        def __init__(self,
                    tmp_file: File):
            File.__init__(self,tmp_file)

In [28]:
d = "tmp_dir.test"

In [95]:
d_tmp = TmpDir(d,True)

In [96]:
d_tmp.tmp_dir

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/tmp_dir.test/tmp_dir_6103'

In [97]:
d_tmp.parent_tmp_dir

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/tmp_dir.test'

In [99]:
d_tmp.mk_tmp_dir()

Temporary directory already exists


In [101]:
d_tmp.rm_tmp_dir(True)

Temporary directory does not exist


In [17]:
class NiiFile(File):
    '''
    class description
    '''
    def __init__(self,
                 file: File):
        File.__init__(self,file)
    
    def rm_ext(self):
        '''
        Intended for use-cases pertaining to NIFTI files.
        '''
        if '.nii.gz' in self.file:
            return self.file[:-7]
        elif '.nii' in self.file:
            return self.file[:-4]
        else:
            return self.file

In [13]:
def test(t: dict={}):
    pass

In [26]:
f = ".txt"
len(f)

4

In [28]:
# f = "../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz"
f = "test.file"

In [29]:
dwi = File(f)

In [31]:
dwi.file

'test.file'

In [32]:
dwi.touch()

'test.file'

In [33]:
os.remove(dwi.file)

In [34]:
dwi.abs_path()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.file'

In [10]:
dwi.rm_ext('.nii.gz')

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi'

In [14]:
'.nii.gz' in dwi.

True

In [142]:
dwi = NiiFile(f)

In [143]:
type(dwi)

__main__.NiiFile

In [144]:
dwi.abs_path()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [145]:
dwi.rm_ext()

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi'

In [147]:
dwi.file

'../sub-C01/ses-001/dwi/bval-b2000_run-01/sub-C01_ses-001_bval-b2000_run-01_dwi.nii.gz'

In [163]:
t = File("test.txt")

In [164]:
t.abs_path()

'/mnt/c/Users/smart/Desktop/IRC317H_NAS/dti/data.dti/xfm_tck/test.txt'

In [110]:
fsl_check = Command("fslmeants")

In [111]:
fsl_check.check_dependency()

In [115]:
q_check = Command("q")

In [116]:
q_check.check_dependency()


 	 The required dependency q is not installed or in the system path. 



DependencyError: System command executable not found.

In [16]:
class LogFile(File):
    '''class doc-string'''
    
    # # Define global variables
    # global log_file
    # global logging
    # global logger
    # global console
    
    def __init__(self,
                 log_file: File):
        '''doc-string'''
        self.log_file: str = log_file
        
        # Set-up logging to file
        logging.basicConfig(level=logging.INFO,
                            format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                            datefmt='%d-%m-%y %H:%M:%S',
                            filename=self.log_file,
                            filemode='a')
        
        # Define a Handler which writes INFO messages or higher to the sys.stderr
        self.console = logging.StreamHandler()
        self.console.setLevel(logging.INFO)
        
        # Add the handler to the root logger
        logging.getLogger().addHandler(self.console)
        
        # Define logging
        self.logger = logging.getLogger(__name__)
        
    
    def log(self,
        log_cmd: str = ""):
        '''
        Log function for logging commands and messages to some log file.

        Usage:
            # Initialize the `log` function command
            log_msg = Command("log")

            # Specify output file and message
            log_msg.log("sub.log","test message 1")

            # Record message, however - no need to re-initialize `log` funcion command or log output file
            log_msg.log("test message 2")

        NOTE: The input `log_file` only needs to be specified once. Once specified,
            this log is written to each time this or the `run` function is invoked.

        Arguments:
            log_file(file): Log file to be written to. 
            log_cmd(str): Message to be written to log file
        '''

        # Log command/message
        self.logger.info(f"{log_cmd}")

In [4]:
log = LogFile("sub-test.log")

In [5]:
log.log("Initial log")

Initial log


In [6]:
log.log("Secon log")

Secon log


In [11]:
type(log.console)

logging.StreamHandler

In [13]:
def test_func(file: File):
    print(f"{file}")

In [14]:
test_func("sub-test.log")

sub-test.log


In [15]:
test_func("file")

file
